In [ ]:
# ================================
# 0) Colab 런타임에 필요한 라이브러리 설치
# ================================
!pip install -q torch torchvision torcheval tqdm
!pip install -q torch torchvision torcheval

# ================================
# 1) 필요한 라이브러리 불러오기
# ================================
import os, random, shutil
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, ConcatDataset, Subset
from torchvision import transforms, datasets, models
from torchvision.transforms import RandAugment
from torchvision.utils import save_image
from torcheval.metrics import MulticlassAccuracy
from tqdm import tqdm

# ================================
# 2) 시드 고정 => 실험의 재현성을 확보하기 위해 사용함.
# ================================
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(42)

# ================================
# 3) 경로 설정
# ================================
original_dir = '/kaggle/input/eyeanemia'  # 원본 이미지
aug_root     = '/content/aug_fold'        # Fold별 증강 이미지
os.makedirs(aug_root, exist_ok=True)

# ================================
# 4) 원본 이미지 리스트 수집
# ================================
all_images, all_labels = [], []
for idx, label in enumerate(sorted(os.listdir(original_dir))):
    folder = os.path.join(original_dir, label)
    for f in os.listdir(folder):
        if f.lower().endswith(('.jpg','jpeg','png')):
            all_images.append(os.path.join(folder, f))
            all_labels.append(idx)

# ================================
# 5) Stratified 5-Fold 세팅
# ================================
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ================================
# 6) Transform 정의
# ================================
# 이미지 정규화를 위한 평균과 표준편차 (일반적으로 ImageNet 통계 사용)
mean, std = [0.485,0.456,0.406], [0.229,0.224,0.225]
# 학습 데이터 전처리 및 기본 증강 파이프라인 정의
#train_transform: 매 Epoch/Batch마다 원본 학습 이미지에 동적으로 적용되는 기본 증강
#val_transform: 검증 이미지에 적용되는 기본적인 전처리 (증강 없음)
#strong_aug: 각 Fold 학습 시작 전, 원본 학습 이미지 일부를 변환하여 새로운 증강 이미지 파일로 저장하고, 이렇게 생성된 증강 이미지를 학습 데이터셋에 추가하는 데 사용되는 강한 증강

# Resize -> CenterCrop -> RandAugment (랜덤 변환 2개 적용, 강도 9) -> RandomHorizontalFlip -> ToTensor -> Normalize
train_transform = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(224), # 이미지 크기 조정
    RandAugment(num_ops=2, magnitude=9),             # RandAugment 적용
    transforms.RandomHorizontalFlip(),               # 좌우 반전
    transforms.ToTensor(), transforms.Normalize(mean,std), # Tensor 변환 및 정규화
])

# 검증 데이터 전처리 파이프라인 정의 (증강 없음)
# Resize -> CenterCrop -> ToTensor -> Normalize
val_transform = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(224), # 이미지 크기 조정
    transforms.ToTensor(), transforms.Normalize(mean,std), # Tensor 변환 및 정규화
])

# Fold별 데이터 증강을 위한 강한 증강 파이프라인 정의
# 이 증강은 학습 루프 시작 전에 별도로 적용하여 추가 학습 데이터를 생성하는 데 사용됩니다.
# Resize -> CenterCrop -> RandomRotation -> RandomHorizontalFlip -> RandomAffine -> ToTensor
strong_aug = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(224), # 이미지 크기 조정
    transforms.RandomRotation(15),                   # 랜덤 회전 (최대 15도)
    transforms.RandomHorizontalFlip(),               # 좌우 반전
    transforms.RandomAffine(5, translate=(0.03,0.03), shear=5), # Affine 변환 (회전, 이동, 전단)
    transforms.ToTensor()                            # Tensor 변환
])

# ================================
# 7) Focal Loss - 데이터셋의 클래스 불균형을 해결하기 위해 사용 -> 쉬운 샘플에 대한 손실 감소, 어려운 샘플에 대한 손실 증가 => 불균형이 있는 데이터셋에서 소수 클래스에 대한 모델의 학습 성능 개선을 위해 넣음
# ================================
class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super().__init__()
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()
    def forward(self, x, y):
        logp = -self.ce(x, y)
        p = torch.exp(logp)
        return -(1 - p) ** self.gamma * logp

# ================================
# 8) 5-Fold 학습
# ================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_fold_accs = []

for fold, (train_idx, val_idx) in enumerate(kf.split(all_images, all_labels), 1):
    print(f"\n===== Fold {fold} =====")

    # 8.1) 원본 데이터셋 분리
    full_orig_train = datasets.ImageFolder(original_dir, transform=train_transform)
    train_orig_ds   = Subset(full_orig_train, train_idx)
    full_orig_val   = datasets.ImageFolder(original_dir, transform=val_transform)
    val_ds          = Subset(full_orig_val, val_idx)

    # 8.2) Fold별 증강 데이터 생성
    fold_aug_dir = os.path.join(aug_root, f"fold{fold}")
    if os.path.exists(fold_aug_dir):
        shutil.rmtree(fold_aug_dir)
    for idx_i in train_idx:
        img_path = all_images[idx_i]
        lbl      = all_labels[idx_i]
        img      = Image.open(img_path).convert('RGB')
        out_cls  = os.path.join(fold_aug_dir, str(lbl))
        os.makedirs(out_cls, exist_ok=True)
        for j in range(5):
            save_image(strong_aug(img),
                       os.path.join(out_cls, f"{idx_i}_{j}.jpg"))

    train_aug_ds = datasets.ImageFolder(fold_aug_dir, transform=train_transform)

    # 8.3) 최종 학습용 데이터셋
    train_ds = ConcatDataset([train_orig_ds, train_aug_ds])

    # 8.4) DataLoader
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=2, pin_memory=True)
    val_loader   = DataLoader(val_ds,    batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

    # 8.5) 모델 정의
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    for name, param in model.named_parameters():
        if not (name.startswith('layer3') or name.startswith('layer4') or name.startswith('fc')):
            param.requires_grad = False
    nf = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.3), nn.Linear(nf,128),
        nn.ReLU(True),   nn.Dropout(0.3),
        nn.Linear(128,2)
    )
    model = model.to(device)

    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                           lr=1e-4, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5)
    criterion = FocalLoss(gamma=2)

    # 8.6) Epoch 루프
    best_acc, patience = 0, 5
    for epoch in range(1, 31):
        # -- Train --
        model.train()
        t_acc = MulticlassAccuracy(num_classes=2).to(device)
        t_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            lam = np.random.beta(0.2,0.2)
            idx2 = torch.randperm(x.size(0))
            mixed_x = lam * x + (1-lam) * x[idx2]
            y_a, y_b = y, y[idx2]

            optimizer.zero_grad()
            out = model(mixed_x)
            loss = lam * criterion(out, y_a) + (1-lam) * criterion(out, y_b)
            loss.backward(); optimizer.step()
            t_loss += loss.item()*x.size(0)
            t_acc.update(out.argmax(1), y)
        scheduler.step()
        train_loss = t_loss / len(train_ds); train_acc = t_acc.compute().item()

        # -- Val --
        model.eval()
        v_acc = MulticlassAccuracy(num_classes=2).to(device)
        v_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                out = model(x)
                loss = criterion(out, y)
                v_loss += loss.item()*x.size(0)
                v_acc.update(out.argmax(1), y)
        val_loss = v_loss/len(val_ds); val_acc = v_acc.compute().item()

        print(f"Epoch {epoch:02d} TL {train_loss:.3f} TA {train_acc:.3f} | "
              f"VL {val_loss:.3f} VA {val_acc:.3f}")

        # -- Early Stopping & Save --
        if val_acc > best_acc:
            best_acc, patience = val_acc, 5
            torch.save(model.state_dict(), f"best_fold{fold}.pth")
        else:
            patience -= 1
            if patience == 0:
                print("Early stopping")
                break

    print(f"Fold {fold} best_val_acc: {best_acc:.3f}")
    best_fold_accs.append(best_acc)

# ================================
# 9) 전체 Fold 평균 출력
# ================================
print(f"\n5-Fold Mean val_acc: {np.mean(best_fold_accs):.3f}")


In [ ]:
# 1. 라이브러리 임포트
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

# 2. 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

# 3. 클래스 레이블 맵핑
# 학습 시 ImageFolder 순서대로 0='Anemia', 1='Non Anemia' 이라 가정
label_map = {0: "Anemia", 1: "Non-Anemia"}

# 4. 모델 정의 (학습과 동일한 구조)
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# layer1,2는 freeze, layer3,4,fc는 fine-tune했던 상태
for name, param in model.named_parameters():
    if not (name.startswith("layer3") or name.startswith("layer4") or name.startswith("fc")):
        param.requires_grad = False

# fc 헤드: Dropout(0.3) → 128 → ReLU → Dropout(0.3) → 2-way Linear
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_ftrs, 128),
    nn.ReLU(inplace=True),
    nn.Dropout(0.3),
    nn.Linear(128, 2)
)

# 5. 학습된 가중치 불러오기
checkpoint_path = "best_fold3.pth"  # 학습 스크립트에서 저장된 가중치
state_dict = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()
print("Model loaded and set to eval mode.")

# 6. 전처리 정의 (검증/추론용)
IMG_SIZE = 224
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# 7. 예측 함수 정의
def predict_image(image_path):
    # 1) 이미지 로드 & 전처리
    image = Image.open(image_path).convert("RGB")
    x = val_transform(image).unsqueeze(0).to(device)  # (1, C, H, W)

    # 2) 추론
    with torch.no_grad():
        logits = model(x)               # (1, 2)
        probs = torch.softmax(logits, dim=1).cpu().squeeze(0)  # (2,)
        conf, pred = torch.max(probs, dim=0)  # 최고 확률과 인덱스

    # 3) 결과 출력
    print(f"Image         : {os.path.basename(image_path)}")
    print(f"Predicted cls : {label_map[pred.item()]}")
    print(f"Confidence    : {conf.item():.4f} ({pred.item()} vs {1-pred.item()})")
    return pred.item(), conf.item()

# 8. 사용 예시
# 테스트할 이미지 경로로 수정하세요
test_image_path = "/content/1000092724.jpg"
predict_image(test_image_path)


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def evaluate_metrics(model, dataloader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            preds = out.argmax(dim=1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1  = f1_score(y_true, y_pred)
    return acc, prec, rec, f1

# --- Fold 루프의 마지막 부분에 삽입 ---
# (train, val dataloader, model 정의 후)
torch.save(model.state_dict(), f"best_fold{fold}.pth")
print(f"Fold {fold} best_val_acc: {best_acc:.3f}")

# 1) 베스트 모델 로드
model.load_state_dict(torch.load(f"best_fold{fold}.pth", map_location=device))

# 2) 원본 검증 세트에 대한 평가
full_orig_val = datasets.ImageFolder(original_dir, transform=val_transform)
val_ds        = Subset(full_orig_val, val_idx)
val_loader    = DataLoader(val_ds, batch_size=32, shuffle=False)

acc, prec, rec, f1 = evaluate_metrics(model, val_loader, device)
print(f"Fold {fold} 평가 지표:")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1 Score : {f1:.4f}")

best_fold_accs.append(best_acc)
